In [1]:
## Initialize ##
!pip install inspyred
import os
# As needed, reset cwd to jupyter notebook location
nb5_path = ("/mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb6_optimizing_ExciteOnly/batch_run_files/")
if os.getcwd() != nb5_path:
    os.chdir(nb5_path)

## USER INPUT REQUIRED ##
## Edit the following parameters as needed ##
## Edit params in the batch_run_files folder as needed (with care) ##
#Evol Batch Config Params
run_name = '5sec_22Mar23_exploratory_100-0' ### Change this to a unique name for the batch run
overwrite_run = False #True will overwrite any existing run with the same name
output_path = nb5_path.replace('batch_run_files', 'output')
run_path = output_path + run_name
method = 'evol'
core_num = 10 #per node
nodes = 1 #number of nodes, or # of cpus or computers
pop_per_core = 2 #this should always be 1
duration_seconds = 5
pop_size = core_num * pop_per_core
max_generations = 2000
skip = True #True will re-run individual batch gens_cands if they already exist

## Prepare Parameter Space for Batch Run ##
from netpyne import specs
#from param_space import get_param_space
from param_space import get_param_space_excite_only
params = get_param_space_excite_only(duration_seconds)
param_space = params.copy()

## Prepare Batch Config
from batch_config import get_batch_config
batch_config = get_batch_config(method, nodes, pop_size, max_generations, run_path, core_num, skip = skip)

#Prepare Batch_Run_Folder and Initial Files
import pickle
import json
import shutil

#make sure batch_run_path does not exist. Avoid overwriting data from any Run
overwrite_run = True #comment out later, this is for debugging
assert overwrite_run or not os.path.exists(run_path), f"Run {run_path} already exists. Set overwrite_run to True to overwrite."
if overwrite_run and os.path.exists(run_path):    
    shutil.rmtree(run_path)

# Create a directory to save the batch files
if not os.path.exists(run_path):
    os.makedirs(run_path)

# Create a directory to save the batch files
with open(f'{run_path}/param_space.pickle', 'wb') as handle:
    pickle.dump(params, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Also save as Json for easy reading
filename = f'{run_path}/param_space.json'
with open(filename, 'w') as file:
    json.dump(params, file, indent=4)

# Save the dictionary as a JSON file
with open(f'{run_path}/batch_config.pickle', 'wb') as f:
    pickle.dump(batch_config, f)

#print
from pprint import pprint
print("run_name: ", run_name)
print("output_path: ", output_path)
print("run_path: ", run_path)
print("pop_size: ", pop_size)
print("core_num: ", core_num)
print("nodes_per_core: ", nodes)
print("pop_per_core: ", pop_per_core)
print("duration_seconds: ", duration_seconds)
print("max_generations: ", max_generations)
print("param_space:")
for key, value in param_space.items():
    print(f"\t{key}: {value}")

 Could not create 
Saving netParams to const_netParams.json ... 
run_name:  5sec_22Mar23_exploratory_100-0
output_path:  /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb6_optimizing_ExciteOnly/output/
run_path:  /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb6_optimizing_ExciteOnly/output/5sec_22Mar23_exploratory_100-0
pop_size:  20
core_num:  10
nodes_per_core:  1
pop_per_core:  2
duration_seconds:  5
max_generations:  2000
param_space:
	duration_seconds: [5, 5]
	probLengthConst: [500, 500]
	propVelocity: [1, 1]
	probEE: [0, 1]
	weightEE: [0.0001, 0.1]
	gnabar_E: [0, 5]
	gkbar_E: [0, 0.5]
	tau1_exc: [0.8, 0.8]
	tau2_exc: [6.0, 6.0]
	stimWeight: [0, 0.002]
	stim_rate: [0, 0.15]
	stim_noise: [0.2, 0.6]


In [2]:
## Run Batch ##
import subprocess
from pprint import pprint
import pickle

# Define the command
cores = core_num
command = f"mpiexec -np {cores} nrniv -mpi batchRun.py"

# Run the command
debug = False
skip = False
if debug:
    from batch_run_files.batchRun import batchRun
	
    # Load the dictionary from the JSON file
    with open('batch_config.pickle', 'rb') as f:
        batch_config = pickle.load(f)
    assert 'method' in batch_config, 'method must be specified in batch_config'
    assert 'skip' in batch_config['runCfg'], 'skip must be specified in batch_config'
    assert 'batchLabel' in batch_config, 'batchLabel must be specified in batch_config'
    #after batch_config is loaded, delete it, there's a copy in the batch_run_path
    #os.remove('batch_config.pickle')
    
    batchRun(
        batchLabel = batch_config['batchLabel'], 
        #method = 'grid', 
        method = batch_config['method'],
        skip = batch_config['runCfg']['skip'],
        batch_config = batch_config
        ) 
else:
    print(command + '\n')
    jobName = f"{run_path}/evol_batchRun"
    process = subprocess.Popen(command.split(' '), stdout=open(jobName + '.run', 'w'), stderr=open(jobName + '.err', 'w'))
    # Get the output and errors, if any
    stdout, stderr = process.communicate()
    # Print the output
    if stdout: pprint(stdout)
    # Print the errors
    if stderr: pprint(stderr)

mpiexec -np 10 nrniv -mpi batchRun.py



KeyboardInterrupt: 

In [ ]:
# import json
# import shutil
# import os
# from batch_run_files.aw_batch_tools import generate_all_figures
# exclude_running = False
# output_only = False
# fresh_figs = False
# for root, dirs, files in os.walk(run_path):
#     for file in files:
#         if file.endswith('_Fitness.json'):
#             if '.archive' in root:
#                 continue
#             if exclude_running and 'output/' in root: continue
#             if output_only and 'output/' not in root: continue
#             specific_output = root.split('optimizing/')[1].split('/gen_')[0]
#             try:
#                 fitness = json.load(open(os.path.join(root, file)))
#                 overall_fitness = fitness['fitness']
#             except:
#                 print(f'Error reading {file}')
            
#             if overall_fitness < 350:
#                 print(f'Overall fitness for {file} is {overall_fitness}')
#                 print(f'Generating plots for {file}')
#                 #move_plots_to_plots_folder(root)
#                 #generate_pdf_reports(root, params)
#                 #print('Plots and pdf reports generated
#             #print(fitness)
#                 batchdata_path = os.path.join(root, file.replace('_Fitness.json', '_data.json'))
#                 assert os.path.exists(batchdata_path), f'{batchdata_path} does not exist'
#                 try:
#                     simLabel = batchdata_path.split('/')[-1].replace('_data.json', '')
#                     run_path = os.path.dirname(os.path.dirname(batchdata_path))
#                     run_basename = os.path.basename(run_path)
#                     print(f'Generating plots for Candidate: {simLabel}, Run: {run_basename}')

#                     # fresh_figs = False
#                     assert os.path.exists(batchdata_path), f'{batchdata_path} does not exist'
#                     cfg_path = batchdata_path.replace('_data.json', '_cfg.json')
#                     assert os.path.exists(cfg_path), f'{cfg_path} does not exist'
#                     fitness_path = batchdata_path.replace('_data.json', '_Fitness.json')
#                     assert os.path.exists(fitness_path), f'{fitness_path} does not exist'

#                     generate_all_figures(
#                         fresh_figs = fresh_figs,
#                         net_activity_params = {'binSize': .03*1000, 
#                                             'gaussianSigma': .12*1000, 
#                                             'thresholdBurst': 1.0},
#                         batchLabel = 'batchRun_evol',
#                         #batchLabel = params['filename'][0],
#                         #minimum peak distance = 0.5 seconds
#                         batch_path = batchdata_path
#                     )

#                     output_path = batchdata_path
#                     output_path = os.path.dirname(output_path)
#                     output_path = f'{output_path}/NetworkBurst_and_Raster_Figs/'
#                     run_grand_path = os.path.dirname(run_path)
#                     plot_row_path = f'{run_grand_path}/goodFit_plots_rows/'
#                     if not os.path.exists(plot_row_path+specific_output):
#                         os.makedirs(plot_row_path+specific_output)
#                     ## shutil copy any files in output_path with the string 'row' in the name to plot path
#                     for rooti, dirsi, filesi in os.walk(output_path):
#                         if '.archive' in rooti: continue
                        
#                         for filei in filesi:
#                             if 'row' in filei:
#                                 if os.path.exists(plot_row_path+filei):
#                                     if fresh_figs: os.remove(plot_row_path+specific_output+'/'+filei)
#                                     else: continue
#                                 shutil.copy(rooti+'/'+filei, plot_row_path+specific_output+'/'+filei)
#                 except:
#                     print(f'Error generating plots for {file}')
#                     continue